In [ ]:
import os
%pwd

In [ ]:
os.chdir('../')
%pwd

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataProcessingConfig:
    training_org_img_folder: Path
    training_segment_img_folder: Path
    testing_org_img_folder: Path
    testing_segment_img_folder: Path
    number_of_classes: int


In [4]:


from src.imageSegmentation.constants import *
from src.imageSegmentation.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_processing_config(self) -> DataProcessingConfig:
        config = self.config.data_preprocessing
        
        data_processing_config = DataProcessingConfig(
            training_org_img_folder = Path(config.training_org_img_folder),
            training_segment_img_folder = Path(config.training_segment_img_folder),
            testing_org_img_folder = Path(config.testing_org_img_folder),
            testing_segment_img_folder = Path(config.testing_segment_img_folder),
            number_of_classes = self.params.CLASSES
        )
        return data_processing_config


In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import cv2
import numpy as np
from tensorflow import keras
from keras.utils import to_categorical

class DataPreprocess:
    def __init__(self, config: DataProcessingConfig):
        self.config = config


    def convert_org_img_to_array(self):
        org_folder = self.config.training_org_img_folder
        x = []
        for file in os.listdir(org_folder):
            path1 = os.path.join(org_folder, file)
            img1 = cv2.imread(path1, 1)
            img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
            img1 = cv2.resize(img1, (512, 512))
            x.append(img1)
        x = np.array(x)
        return x

    def convert_seg_img_to_array(self):
        seg_folder = self.config.training_segment_img_folder
        y = []
        for file2 in os.listdir(seg_folder):
            path2 = os.path.join(seg_folder, file2)
            img2 = np.array(cv2.imread(path2, 1))
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
            img2 = cv2.resize(img2, (512, 512))
            y.append(img2)
        y = np.array(y)
        return y


    @staticmethod
    def rgb_to_2D_label(image):
        image_seg = np.zeros(image.shape, dtype= np.uint8)

        image_seg[np.all(image == np.array([255, 235, 0]), axis= -1)] = 0
        image_seg[np.all(image == np.array([0, 0 ,0]), axis= -1)] = 1
        image_seg[np.all(image == np.array([27 ,71, 151]), axis= -1)] = 2
        image_seg[np.all(image == np.array([111, 48, 253]), axis= -1)] = 3
        image_seg[np.all(image == np.array([137, 126, 126]), axis= -1)] = 4
        image_seg[np.all(image == np.array([201, 19, 223]), axis= -1)] = 5
        image_seg[np.all(image == np.array([254, 233, 3]), axis= -1)] = 6
        image_seg[np.all(image == np.array([255, 0, 29]), axis= -1)] = 7
        image_seg[np.all(image == np.array([255, 159, 0]), axis= -1)] = 8
        image_seg[np.all(image == np.array([255, 160, 1]), axis= -1)] = 9

        image_seg = image_seg[:,:,0]
        return image_seg
    
    
    def rgb_to_class_num(self):
        segArray = self.convert_seg_img_to_array()
        labels_list = []                             
        for i in range(segArray.shape[0]):
            label = self.rgb_to_2D_label(segArray[i])
            labels_list.append(label)
        labels_arr = np.array(labels_list)
        labels_arr = np.expand_dims(labels_arr, axis=3)
        return labels_arr
    
    def convert_to_categorical(self):
        imgArray = self.rgb_to_class_num()
        classes = self.config.number_of_classes
        label_categorical = to_categorical(imgArray, num_classes = classes)
        return label_categorical

[2023-07-26 23:13:10,313: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-07-26 23:13:10,315: INFO: utils: NumExpr defaulting to 8 threads.]


In [6]:

try:
    config = ConfigurationManager()
    data_process_config = config.get_data_processing_config()
    data_process = DataPreprocess(config = data_process_config)
    data_process.convert_org_img_to_array()
    data_process.convert_seg_img_to_array()
    data_process.rgb_to_class_num()
    data_process.convert_to_categorical()
except Exception as e:
    raise e

[2023-07-26 23:13:10,660: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-26 23:13:10,662: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-26 23:13:10,663: INFO: common: created directory at: artifacts]


# Extract unique channels from array

In [ ]:
import os
import cv2
import numpy as np


x = []
fl = "C:\\Users\\jites\\Desktop\\DL_model_deployment\\image-classification-project\\artifacts\\data_ingestion\\images\\Training_segment"
for file in os.listdir(fl):
    path1 = os.path.join(fl, file)
    img1 = cv2.imread(path1, 1)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
#     img1 = cv2.resize(img1, (512, 512))
    x.append(img1)
    
x = np.array(x)

In [ ]:
import cv2
import numpy as np

unique_colors = []

for i in range(0, x.shape[0]):
    flat_array = x[i].reshape(-1, 3)
    
    unique_colors_i = np.unique(flat_array, axis = 0)
    unique_colors.extend(unique_colors_i)
unique_colors = np.array(unique_colors)
# print(unique_colors)
final_unique_colors = np.unique(unique_colors, axis=0)

for color in final_unique_colors:
    print(color)


#     unique_colors_i = np.unique(flat_array, axis=0)
#     unique_colors.extend(unique_colors_i)

# unique_colors = np.array(unique_colors)
# final_unique_colors = np.unique(unique_colors, axis=0)

# for color in final_unique_colors:
#     print(color)

# Extract unique channels from image

In [ ]:
import cv2
import numpy as np

unique_colors = []

fl = "C:\\Users\\jites\\Desktop\\DL_model_deployment\\image-classification-project\\artifacts\\data_ingestion\\images\\Training_segment\\"


for i,name in enumerate(os.listdir(fl)):
    image = cv2.imread(fl + name)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_array = np.array(image)
    flat_array = image_array.reshape(-1, 3)
    unique_colors_i = np.unique(flat_array, axis=0)
    unique_colors.extend(unique_colors_i)

unique_colors = np.array(unique_colors)
final_unique_colors = np.unique(unique_colors, axis=0)

for color in final_unique_colors:
    print(color)

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



In [ ]:
def add_numbers(a: int, b: int) -> int:
    return a + b

add_numbers(34,53)
type(add_numbers(34,53))

In [ ]:
def add(a: int, b: int) -> str:
    return a + b

add(34,53)
type(add(34,53))